In [8]:
import tensorflow as tf 
import numpy as np
import random 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from scipy.spatial.distance import euclidean
import logging
import os

In [9]:
D = 19
N = 20

T_max =  10

NConv_min =  1
NConv_max =  3

NFil_min =  3
NFil_max =  256

SKer_min =  3
SKer_max =  9

SPool_min =  1
SPool_max =  3

SStr_min =  1
SStr_max =  2

NFc_min =  1
NFc_max =  2

NNeu_min =  1
NNeu_max =  300

S_batch =  128

epochs_fitness =  1
epochs_full_training  = 100
R_L  = 0.001

In [10]:
def apply_constraints(offspring_vec,
                      NConv_min, NConv_max,
                      NFil_min, NFil_max,
                      SKer_min, SKer_max,
                      SPool_min, SPool_max,
                      SStr_min, SStr_max,
                      NFc_min, NFc_max,
                      NNeu_min, Nneu_max):
    logging.info("doing boundry check  and rounding integer of new offspring generated in teacher phase")
    D = len(offspring_vec)
    
    #N_conv
    offspring_vec[0] = np.clip(offspring_vec[0], NConv_min, NConv_max)
    # Conv layers
    for l in range(1, NConv_max + 1):
        offspring_vec[2*l - 1] = np.clip(offspring_vec[2*l - 1], NFil_min, NFil_max)
        offspring_vec[2*l] = np.clip(offspring_vec[2*l], SKer_min, SKer_max)
    #Pool layers
    base_idx = 2 * NConv_max
    for l in range(1, NConv_max + 1):
        offspring_vec[base_idx + 3*l - 2] = np.clip(offspring_vec[base_idx + 3*l - 2], 0, 1) # P_pool
        offspring_vec[base_idx + 3*l - 1] = np.clip(offspring_vec[base_idx + 3*l - 1], SPool_min, SPool_max)
        offspring_vec[base_idx + 3*l] = np.clip(offspring_vec[base_idx + 3*l], SStr_min, SStr_max)
    # N_fc
    fc_base_idx = 5 * NConv_max + 1
    offspring_vec[fc_base_idx] = np.clip(offspring_vec[fc_base_idx], NFc_min, NFc_max)
    # FC layers
    for q in range(1, NFc_max + 1):
        offspring_vec[fc_base_idx + q] = np.clip(offspring_vec[fc_base_idx + q],NNeu_min, Nneu_max)
        
    #rounding
    offspring_vec[0] = round(offspring_vec[0])
    for l in range(1, NConv_max + 1):
        offspring_vec[2*l - 1] = round(offspring_vec[2*l - 1])
        offspring_vec[2*l] = round(offspring_vec[2*l])
    base_idx = 2 * NConv_max
    for l in range(1, NConv_max + 1):
        #do NOT round the P_pool probability
        offspring_vec[base_idx + 3*l - 1] = round(offspring_vec[base_idx + 3*l - 1])
        offspring_vec[base_idx + 3*l] = round(offspring_vec[base_idx + 3*l])
    fc_base_idx = 5 * NConv_max + 1
    offspring_vec[fc_base_idx] = round(offspring_vec[fc_base_idx])
    for q in range(1, NFc_max + 1):
        offspring_vec[fc_base_idx + q] = round(offspring_vec[fc_base_idx + q])
        
    return offspring_vec


In [11]:
def get_boundary_arrays(NConv_min, NConv_max,
    NFil_min, NFil_max,
    SKer_min, SKer_max,
    SPool_min, SPool_max,
    SStr_min, SStr_max,
    NFC_min, NFC_max,
    NNeu_min, NNeu_max):
    logging.info("xmin and xmax initialize")
    D = 5 * NConv_max + NFC_max + 2
    X_min = np.zeros(D)
    X_max = np.zeros(D)
    # N_conv
    X_min[0], X_max[0] = NConv_min, NConv_max
    # Conv layers
    for l in range(1, NConv_max + 1):
        X_min[2*l - 1], X_max[2*l - 1] = NFil_min, NFil_max
        X_min[2*l], X_max[2*l] = SKer_min, SKer_max
    # Pool layers
    base_idx = 2 * NConv_max
    for l in range(1, NConv_max + 1):
        X_min[base_idx + 3*l - 2], X_max[base_idx + 3*l - 2] = 0.0, 1.0
        X_min[base_idx + 3*l - 1], X_max[base_idx + 3*l - 1] = SPool_min, SPool_max
        X_min[base_idx + 3*l], X_max[base_idx + 3*l] = SStr_min, SStr_max
    # N_fc
    fc_base_idx = 5 * NConv_max + 1
    X_min[fc_base_idx], X_max[fc_base_idx] = NFC_min, NFC_max
    # FC layers
    for q in range(1, NFC_max + 1):
        X_min[fc_base_idx + q], X_max[fc_base_idx + q] = NNeu_min, NNeu_max
    logging.info(f"xmax {X_max}\nx min  {X_min}")
    return X_min, X_max

In [12]:
def self_learning_schema(learner,teacher,
                        X_max, X_min,
                        NConv_min, NConv_max,
                        NFil_min, NFil_max,
                        SKer_min, SKer_max,
                        SPool_min, SPool_max,
                        SStr_min, SStr_max,
                        NFc_min, NFc_max,
                        NNeu_min, Nneu_max,
                        R_train, R_valid,S_batch = 128, ε_train = 1,
                        R_L = 0.001, C_num = 10,
                        input_shape =  (28,28,1)
                        ):
    D = len(learner["vector"])
    Xn = learner["vector"].copy()
    d_rand = random.randint(0, D - 1)
    r5 = random.uniform(-1, 1)
    Xn[d_rand] += r5 * (X_max[d_rand] - X_min[d_rand])
    Xn = apply_constraints(Xn,
                NConv_min, NConv_max,
                NFil_min, NFil_max,
                SKer_min, SKer_max,
                SPool_min, SPool_max,
                SStr_min, SStr_max,
                NFc_min, NFc_max,
                NNeu_min, Nneu_max )
    
    fitness_self_learner = evaluate_fitness(Xn, R_train, R_valid, NConv_max,S_batch = 128, ε_train = 1,
                                    R_L = 0.001, C_num = 10,
                                    input_shape =  (28,28,1))
    if fitness_self_learner < learner['fitness']:
        updated_learner = {"vector": Xn, "fitness": fitness_self_learner}
        if fitness_self_learner < teacher['fitness']:
            teacher['vector'] = Xn.copy()
            teacher['fitness'] = fitness_self_learner
            logging.info(f"New Teacher found via Self-Learning Fitness: {fitness_self_learner:.4f} ***")
        return updated_learner,teacher
    else:
        return learner, teacher


def adaptive_peer_learning(n, P_off, teacher,NConv_min, NConv_max,
                                NFil_min, NFil_max,
                                SKer_min, SKer_max,
                                SPool_min, SPool_max,
                                SStr_min, SStr_max,
                                NFc_min, NFc_max,
                                NNeu_min, Nneu_max,R_train,R_valid,S_batch = 128, ε_train = 1,
                                R_L = 0.001, C_num = 10,
                                input_shape =  (28,28,1)):
    N = len(P_off)
    D = len(P_off[0]['vector'])
    learner = P_off[n]
    Xn = learner['vector'].copy()
    rank = (N - 1) - n
    p_n_PL = rank / N
    peer_indices = list(range(N))
    peer_indices.remove(n)
    p, s, u = random.sample(peer_indices, 3)
    peer_p, peer_s, peer_u = P_off[p]['vector'], P_off[s]['vector'], P_off[u]['vector']
    chi_n = random.uniform(0.5, 1.0)
    for d in range(D):
        if random.random() < p_n_PL:
            Xn[d] = peer_p[d] + chi_n * (peer_s[d] - peer_u[d])
    Xn = apply_constraints(Xn,
                NConv_min, NConv_max,
                NFil_min, NFil_max,
                SKer_min, SKer_max,
                SPool_min, SPool_max,
                SStr_min, SStr_max,
                NFc_min, NFc_max,
                NNeu_min, Nneu_max )
    
    fitness_adaptive_learner = evaluate_fitness(Xn, R_train, R_valid, NConv_max,S_batch = 128, ε_train = 1,
                                    R_L = 0.001, C_num = 10,
                                    input_shape =  (28,28,1))
    if fitness_adaptive_learner < learner['fitness']:
        updated_learner = {"vector": Xn, "fitness": fitness_adaptive_learner}
        if fitness_adaptive_learner < teacher['fitness']:
            teacher['vector'] = Xn.copy()
            teacher['fitness'] = fitness_adaptive_learner
            logging.info(f"New Teacher found via Self-Learning Fitness: {fitness_adaptive_learner:.4f} ***")
        return updated_learner,teacher
    else:
        return learner, teacher

In [ ]:
def initialize_population(
    N,  # population size
    NConv_min, NConv_max,
    NFil_min, NFil_max,
    SKer_min, SKer_max,
    SPool_min, SPool_max,
    SStr_min, SStr_max,
    NFC_min, NFC_max,
    NNeu_min, NNeu_max,
    R_train, R_valid
):
    logging.info("Initializing population")
    D = 5 * NConv_max + NFC_max + 2
    teacher_solution = {
        "vector": None,
        "fitness": float('inf')
    }
    population = []
    for n in range(N):
        Xn = np.zeros(D)
        NConv = random.randint(NConv_min, NConv_max) #initializing eith number of learners
        Xn[0] = NConv
        for l in range(1,NConv_max+1):
            NFil = random.randint(NFil_min, NFil_max)
            SKer = random.randint(SKer_min, SKer_max)
            Xn[2*l - 1] = NFil
            Xn[2*l] = SKer
        for l in range(1,NConv_max+1):
            PPool = random.uniform(0, 1)
            SPool = random.randint(SPool_min, SPool_max)
            SStr = random.randint(SStr_min, SStr_max)
            base_idx = 2 * NConv_max
            Xn[base_idx+3*l-2] = PPool
            Xn[base_idx+3*l-1] = SPool
            Xn[base_idx+3*l] = SStr
        fc_base_idx = 5 * NConv_max + 1
        Xn[fc_base_idx]= random.randint(NFC_min, NFC_max)
        for q in range(1,NFC_max+1):
            Xn[fc_base_idx + q]= random.randint(NNeu_min, NNeu_max)
        
        fitness = evaluate_fitness(Xn,R_train, R_valid,NConv_max)

        learner_solution = {
            "vector": Xn,
            "fitness": fitness
        }
        population.append(learner_solution)
        logging.info(f"Learner {n+1} initialized with fitness: {fitness:.4f}")
        if fitness < teacher_solution["fitness"]:
            teacher_solution["vector"] = Xn.copy()
            teacher_solution["fitness"] = fitness
            logging.info(f"*** New Teacher found! Fitness: {fitness:.4f} ***")
    return population, teacher_solution
        
        
def evaluate_fitness(
    Xn,R_train, R_valid,NConv_max,
    S_batch = 128, ε_train = 1,
    R_L = 0.001, C_num = 10,
    input_shape =  (28,28,1)
):
    logging.info("Evaluating Fitness")
    try:
        # logging.info("trying statement")
        model = decode_learner_to_cnn(
                Xn, NConv_max,
                input_shape,
                C_num
            )
        # logging.info(1)
        optimizer = Adam(learning_rate=R_L)
        model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        # logging.info(2)
        x_train, y_train = R_train
        x_val, y_val = R_valid
        model.fit(
                x_train, y_train,
                epochs=ε_train,
                batch_size=S_batch,
                validation_data=R_valid,
                verbose=0  
            )
        # logging.info(3)
        
        loss, accuracy = model.evaluate(x_val, y_val, verbose=0)
        # The fitness is simply the error
        classification_error = 1.0 - accuracy
        return classification_error
    
    except Exception as e:
        logging.info(f"Error in fitness evaluation")
        return float('inf')


def decode_learner_to_cnn(Xn, NConv_max,
                          input_shape =  (28,28,1),C_num = 10
            ):
    logging.info("making cnn(decoding xn to cnn model)")
    model = Sequential()
    model.add(tf.keras.Input(shape=input_shape))
    NConv = int(Xn[0])
    NFC = int(Xn[5 * NConv_max +1])
    for l in range(1,NConv+1):
        NFil = int(Xn[2 * l-1])
        SKer = int(Xn[2 * l])
        model.add(Conv2D(
                filters=NFil,
                kernel_size=(SKer, SKer),
                activation='relu',
                padding='valid',
                kernel_initializer='he_normal' 
            ))
        model.add(BatchNormalization())
        pooling_type_d = 2 * NConv_max + 3 * l - 2
        kernal_size_of_pooling_layer = 2 * NConv_max + 3 * l - 1
        stride_size_of_polling_layer = 2 * NConv_max + 3 * l 
        PPool = Xn[pooling_type_d]
        SPool = int(Xn[kernal_size_of_pooling_layer])
        SStr = int(Xn[stride_size_of_polling_layer])
        if 0 <= PPool < 1/3:
            pass  # No pooling
        elif 1/3 <= PPool < 2/3:
            model.add(MaxPooling2D(pool_size=(SPool, SPool), strides=(SStr, SStr)))
        else:  # PPool ≥ 2/3
            model.add(AveragePooling2D(pool_size=(SPool, SPool), strides=(SStr, SStr)))
        
    model.add(Flatten())
    
    for q in range(1, NFC + 1):
        dimension_of_fully_connected_layer = (5 * NConv_max + 1) + q
        num_neurons = int(Xn[dimension_of_fully_connected_layer])
        model.add(Dense(num_neurons, activation='relu'))
        model.add(Dropout(0.5))
    model.add(Dense(C_num, activation='softmax'))
    return model
        
        
def modified_teacher_phase(population, teacher, R_train, R_valid,
                           S_batch , ε_train, R_L, C_num,
                           NConv_max,NConv_min,
                           NFil_min, NFil_max,
                           SKer_min, SKer_max,
                           SPool_min, SPool_max,
                           SStr_min, SStr_max,
                           NFc_min, NFc_max,
                           NNeu_min, Nneu_max 
                           ):
    # population => [learner_solution = {
        #     "vector": Xn,
        #     "fitness": fitness
        # }]  -- at ith is knows as learner
    # teacher_solution => teacher_solution = {
        # "vector": None,             #best vector(Xn)
    #     "fitness": float('inf')     # best fitness
    # }
    logging.info("Starting modified teacher phase")
    sorted_population = sorted(population, key=lambda x: x['fitness'], reverse=True)
    N = len(sorted_population)
    D = len(sorted_population[0]['vector'])
    unique_means = []
    for n in range(N):
        fitter_learners = sorted_population[n:]
        mean_vector = np.mean([learner['vector'] for learner in fitter_learners], axis=0)
        unique_means.append(mean_vector)
    
    unique_social_exemplars = []
    for n in range(N):
        if n == N - 1:
            unique_social_exemplars.append(None)
            continue
        exemplar_vector = np.zeros(D)
        for d in range(D):
            better_learner_idx = random.randint(n + 1, N - 1)
            exemplar_vector[d] = sorted_population[better_learner_idx]['vector'][d]
        unique_social_exemplars.append(exemplar_vector)
    
    logging.info("done making unique and social exemplar")
    
    P_off = []
    current_teacher = teacher.copy()
    for n in range(N):
        learner = sorted_population[n]
        if n == N - 1:
            P_off.append(learner)
            logging.info(f"Best learner {n+1} carried over. Fitness: {learner['fitness']:.4f}")
            continue
        
        logging.info(f"Processing Learner {n+1}/{N}...")
        r3, r4 = random.random(), random.random()
        F_T = random.choice([1, 2])
        teacher_vec = current_teacher['vector']
        learner_vec = learner['vector']
        mean_vec = unique_means[n]
        social_exemplar_vec = unique_social_exemplars[n]
        # Xn_Off = learner_vec + r3*(teacher_vec - F_T * mean_vec) + r4*(social_exemplar_vec - learner_vec)
        offspring_vector = (learner_vec + 
                            r3 * (teacher_vec - F_T * mean_vec) + 
                            r4 * (social_exemplar_vec - learner_vec))
        
    
        """Applying Constraints = >
            core formula in the Teacher Phase:
            offspring_vector = (learner_vec +
                                r3 * (teacher_vec - F_T * mean_vec) +
                                r4 * (social_exemplar_vec - learner_vec))
            This is just vector math. The result of these additions and subtractions is guaranteed to produce numbers that are not clean
            simple analogy: Imagine you are designing a car, and the number of doors must be an integer between 2 and 5. You have two designs, one with 2 doors and one with 5. If you average them, you get 3.5 doors. This is a mathematically correct, but you can't build a car with 3.5 doors.
            The apply_constraints function is the engineer who looks at the "3.5 doors" calculation and says, That's not valid. I'll round that to 4 doors
        """
        offspring_vector = apply_constraints(offspring_vector,
                      NConv_min, NConv_max,
                      NFil_min, NFil_max,
                      SKer_min, SKer_max,
                      SPool_min, SPool_max,
                      SStr_min, SStr_max,
                      NFc_min, NFc_max,
                      NNeu_min, Nneu_max 
                      )
        offspring_fitness = evaluate_fitness(offspring_vector, R_train, R_valid, NConv_max, S_batch, ε_train, R_L, C_num)
        offspring = {"vector": offspring_vector, "fitness": offspring_fitness}
        P_off.append(offspring)
        logging.info(f"""
              before implementing teacher phase learner fitness :{learner['fitness']}
              Offspring created with fitness: {offspring_fitness:.4f}""")
        if offspring_fitness < current_teacher['fitness']:
            current_teacher['vector'] = offspring_vector.copy()
            current_teacher['fitness'] = offspring_fitness
            logging.info(f"  -> *** New Teacher found! Fitness: {offspring_fitness:.4f} ***")
    
    return P_off, current_teacher


def modified_learner_phase(P_off,teacher,R_train, R_valid,
                            X_max, X_min,
                            NConv_min, NConv_max,
                            NFil_min, NFil_max,
                            SKer_min, SKer_max,
                            SPool_min, SPool_max,
                            SStr_min, SStr_max,
                            NFc_min, NFc_max,
                            NNeu_min, Nneu_max,
                            S_batch = 128, ε_train = 1,
                            R_L = 0.001, C_num = 10,
                            input_shape =  (28,28,1)):
    logging.info("Modified_learner_pahse")
    P_off = sorted(P_off,key = lambda x:x["fitness"], reverse = True)
    N = len(P_off)
    D = len(P_off[0]['vector'])
    current_teacher = teacher.copy()
    updated_offspring_population = []
    for n in range(N):
        learner = P_off[n]
        logging.info(f"Processing Learner {n+1}/{N}")
        r6 = random.random()
        P_SL = 1/D
        if r6<P_SL:
            logging.info("Performing Self-Learning")
            updated_learner, current_teacher = self_learning_schema(learner,current_teacher,
                        X_max, X_min,
                        NConv_min, NConv_max,
                        NFil_min, NFil_max,
                        SKer_min, SKer_max,
                        SPool_min, SPool_max,
                        SStr_min, SStr_max,
                        NFc_min, NFc_max,
                        NNeu_min, Nneu_max,
                        R_train, R_valid,S_batch, ε_train = 1,
                        R_L = 0.001, C_num = 10,
                        input_shape =  (28,28,1)
                        )
        else:
            logging.info("Performing Adaptive lerning")
            updated_learner, current_teacher = adaptive_peer_learning(n, P_off, current_teacher,NFil_min, NFil_max,
                                                    SKer_min, SKer_max,
                                                    SPool_min, SPool_max,
                                                    SStr_min, SStr_max,
                                                    NFc_min, NFc_max,
                                                    NNeu_min, Nneu_max,R_train,R_valid,S_batch = 128, ε_train = 1,
                                                    R_L = 0.001, C_num = 10,
                                                    input_shape =  (28,28,1))
        updated_offspring_population.append(updated_learner)
    return updated_offspring_population, current_teacher


def dual_criterion(N, current_pop, offspring_pop):
    logging.info("dual_criterion")
    merged_population = current_pop + offspring_pop
    merged_population.sort(key=lambda x: x['fitness'])
    
    best_vector = merged_population[0]['vector']
    for learner in merged_population:
        learner['diversity'] = euclidean(learner['vector'], best_vector)
    K = random.randint(1, N)
    next_generation = merged_population[:K]
    candidates = merged_population[K:]
    if not candidates:
        return next_generation
    fitness_values = [c['fitness'] for c in candidates]
    diversity_values = [c['diversity'] for c in candidates]
    F_min, F_max = min(fitness_values), max(fitness_values)
    Dis_min, Dis_max = min(diversity_values), max(diversity_values)
    for _ in range(N - K):
        tourn_a, tourn_b = random.sample(candidates, 2)
        alpha = np.random.normal(0.9, 0.05)
        alpha = np.clip(alpha, 0.8, 1.0)
        f_range = F_max - F_min if F_max > F_min else 1
        d_range = Dis_max - Dis_min if Dis_max > Dis_min else 1
        wf_a = alpha * ((tourn_a['fitness'] - F_min) / f_range) + (1 - alpha) * ((Dis_max - tourn_a['diversity']) / d_range)
        wf_b = alpha * ((tourn_b['fitness'] - F_min) / f_range) + (1 - alpha) * ((Dis_max - tourn_b['diversity']) / d_range)
        winner = tourn_a if wf_a <= wf_b else tourn_b
        next_generation.append(winner)
        candidates.remove(winner)
    return next_generation
    

def load_dataset(dataset_name='mnist'):
    if dataset_name.lower() == 'mnist':
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    elif dataset_name.lower() == 'fashion_mnist':
        (x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
    else:
        raise ValueError(f"Dataset '{dataset_name}' is not supported by this simple loader.")

    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255
    
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)
    num_classes = 10
    y_train = tf.keras.utils.to_categorical(y_train, num_classes)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes)
    
    R_train = (x_train, y_train)
    R_valid = (x_test, y_test)
    return R_train, R_valid, x_train.shape[1:], num_classes


if __name__ == '__main__':
    log_dir = "loging"
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    log_file = os.path.join(log_dir, 'result.log')
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file, mode='w'), 
            logging.StreamHandler() 
        ]
    )
    ε_train = epochs_fitness
    R_train, R_valid, input_shape, C_num = load_dataset('fashion_mnist')
    X_min, X_max = get_boundary_arrays(NConv_min, NConv_max,
    NFil_min, NFil_max,
    SKer_min, SKer_max,
    SPool_min, SPool_max,
    SStr_min, SStr_max,
    NFc_min, NFc_max,
    NNeu_min, NNeu_max)
    current_population, teacher = initialize_population(
        N,  # population size
        NConv_min, NConv_max,
        NFil_min, NFil_max,
        SKer_min, SKer_max,
        SPool_min, SPool_max,
        SStr_min, SStr_max,
        NFc_min, NFc_max,
        NNeu_min, NNeu_max,
        R_train, R_valid
        )
    for t in range(T_max):
        logging.info(f"\nStarting Generation {t+1}/{T_max}")
        offspring_after_teacher, teacher = modified_teacher_phase(current_population, teacher, R_train, R_valid,
                           S_batch , ε_train, R_L, C_num,
                           NConv_max,NConv_min,
                           NFil_min, NFil_max,
                           SKer_min, SKer_max,
                           SPool_min, SPool_max,
                           SStr_min, SStr_max,
                           NFc_min, NFc_max,
                           NNeu_min, NNeu_max
        )
        offspring_after_learner, teacher = modified_learner_phase(
            offspring_after_teacher,teacher,R_train, R_valid,
            X_max, X_min,
            NConv_min, NConv_max,
            NFil_min, NFil_max,
            SKer_min, SKer_max,
            SPool_min, SPool_max,
            SStr_min, SStr_max,
            NFc_min, NFc_max,
            NNeu_min, NNeu_max,
            S_batch, ε_train,
            R_L , C_num,
            input_shape 
        )
        current_population = dual_criterion(N, current_population, offspring_after_learner)
        current_population.sort(key=lambda x: x['fitness'])
        if current_population[0]['fitness'] < teacher['fitness']:
            teacher = current_population[0].copy()
            logging.info(f"Teacher updated after selection. New best fitness: {teacher['fitness']:.4f}")
    
    logging.info("\n MTLBORKS-CNN Search Complete")
    logging.info(f"Final best fitness found (Teacher): {teacher['fitness']:.4f}")
    logging.info("Starting full training on the best discovered architecture")
    final_model = decode_learner_to_cnn(teacher['vector'], NConv_max, input_shape,C_num)
    final_model.compile(optimizer=Adam(learning_rate=R_L), loss='categorical_crossentropy', metrics=['accuracy'])
    
    final_model.fit(
        R_train[0], R_train[1],
        batch_size=S_batch,
        epochs=epochs_full_training,
        validation_data=R_valid,
        verbose=1 
    )
    
    final_loss, final_accuracy = final_model.evaluate(R_valid[0], R_valid[1], verbose=0)
    logging.info("\n--- Final Model Performance ---")
    logging.info(f"Final Accuracy: {final_accuracy * 100:.2f}%")
    logging.info(f"Final Classification Error: {1 - final_accuracy:.4f}")
    final_model.summary()

                

    
    
    
    
                                                   
            
            
    

        

        
    
    
    
    
    
        
    
    
        
            
            
            
            
        
        
        
    

2025-08-22 13:34:05,953 - INFO - xmin and xmax initialize
2025-08-22 13:34:05,979 - INFO - xmax [  3. 256.   9. 256.   9. 256.   9.   1.   3.   2.   1.   3.   2.   1.
   3.   2.   2. 300. 300.]
x min  [1. 3. 3. 3. 3. 3. 3. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1.]
2025-08-22 13:34:05,981 - INFO - Initializing population
2025-08-22 13:34:05,985 - INFO - Evaluating Fitness
2025-08-22 13:34:05,986 - INFO - making cnn(decoding xn to cnn model)
2025-08-22 13:39:44,080 - INFO - Learner 1 initialized with fitness: 0.2672
2025-08-22 13:39:44,124 - INFO - *** New Teacher found! Fitness: 0.2672 ***
2025-08-22 13:39:44,126 - INFO - Evaluating Fitness
2025-08-22 13:39:44,127 - INFO - making cnn(decoding xn to cnn model)
2025-08-22 13:43:18,837 - INFO - Learner 2 initialized with fitness: 0.5895
2025-08-22 13:43:18,839 - INFO - Evaluating Fitness
2025-08-22 13:43:18,840 - INFO - making cnn(decoding xn to cnn model)
2025-08-22 13:46:25,126 - INFO - Learner 3 initialized with fitness: 0.1466
2025-08-22 13